In [1]:
import cv2, sys
import socket #실시간통신
import threading 
import time
import datetime

tello_address = ('192.168.10.1', 8889) #두가지요소를 가지고 있는 튜플
local_address = ('', 9000)

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) #소켓통신
sock.bind(local_address) #소켓을 local address에 bind

#Tello라는 드론에게 딜레이 줘서 메세지 보내는 함수
def send(message, delay=0):
  try:
    sock.sendto(message.encode(), tello_address)
    print("Sending message: " + message)
  except Exception as e:
    print("Error sending: " + str(e))
    
  time.sleep(delay)

#데이터를 받는 함수
def receive():
  while True:
    try:
      response, ip_address = sock.recvfrom(128)
      print("Received message: " + response.decode(encoding='utf-8'))
    except Exception as e:
      sock.close()
      print("Error receiving: " + str(e))
      break
      
receiveThread = threading.Thread(target=receive)
receiveThread.daemon = True
receiveThread.start()

send("command", 5) #드론한테 명령
send("streamon", 5) #카메라on

capture = cv2.VideoCapture(0) # 텔로 EDU 캠 포트

while True: 
    ret, frame = capture.read()
    if ret == True:
        cv2.imshow('frame', frame)

    key = cv2.waitKey(1);
    if key == ord('q'):
        send("streamoff")
        break
    elif key == ord('s'):
        file = datetime.datetime.now().strftime("%Y%m%d_%H%M%S%f") +'.jpg' #중복 안되게 시간을 파일 이름으로 씀
        cv2.imwrite(file,frame)  
        print(file, ' saved')

        
capture.release()
cv2.destroyAllWindows()


Sending message: command
Sending message: streamon
Sending message: streamoff
